In [171]:
raw_file = r"D:\github\backorder_prediction_model\dataset\backorder.csv.gz"
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
df = pd.read_csv(raw_file , compression='gzip')
df.drop(['Unnamed: 0', 'sku'] , axis = 1 ,inplace =True)
df['went_on_backorder'] = LabelEncoder().fit_transform(df['went_on_backorder'])

C:\Users\nitin\AppData\Local\Temp\ipykernel_9696\1801431518.py:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_file , compression='gzip')


In [172]:
df.shape

(1929937, 22)

In [173]:
X = df.drop('went_on_backorder' , axis =1 )
y = df['went_on_backorder']

In [178]:
data_train, data_temp, labels_train, labels_temp = train_test_split(X,y, test_size=0.05, random_state=42)

# Step 2: Split the temporary set into development and test sets (50% temp, 50% test)
X_dev, X_test, y_dev, y_test = train_test_split(data_temp, labels_temp, test_size=0.5, random_state=42)

In [179]:
X_dev.shape

(48248, 21)

In [190]:
print(X_dev.shape)
len(y_dev[y_dev==1])

(48248, 21)


358

In [200]:
import pandas as pd
import random


# Important features by RFE
imp_features = ['forecast_6_month', 'forecast_9_month', 'sales_9_month','pieces_past_due', 'perf_6_month_avg',
                'national_inv', 'in_transit_qty', 'sales_3_month','pieces_past_due']

# Function to perform data augmentation with modifications
def augment_data(X_train , y_train, noise_range=1):
    temp = pd.DataFrame(y_train , columns=['went_on_backorder'])
    new_df = pd.concat([X_train , temp], axis=1)

    # Filter rows where 'wen_on_backorder' is 1
    backorder_df = new_df[new_df['went_on_backorder'] == 1]
    
    # Check if there are any 'No' cases
    if len(new_df[new_df['went_on_backorder'] == 0]) == 0:
        return new_df  # If no 'No' cases, return the original DataFrame
    
    # Calculate the number of times we need to duplicate to balance the classes
    num_times = len(new_df[new_df['went_on_backorder'] == 0]) // len(backorder_df)
    
    # Duplicate the rows with 'wen_on_backorder' = 1
    augmented_df = pd.concat([new_df] + [backorder_df] * num_times, ignore_index=True , axis=0)
    
    # Apply modifications to the duplicated rows
    for col in imp_features:
        # You can customize the modification logic as per your requirement
        augmented_df.loc[augmented_df['went_on_backorder'] == 1, col] += random.uniform(-noise_range, noise_range)
    
    
    return augmented_df.drop('went_on_backorder' , axis=1) , augmented_df['went_on_backorder']

# Call the function to augment the data with modifications
X_dev , y_dev = augment_data(X_dev , y_dev)


# Display the class counts in the augmented DataFrame
print(len(y_dev[y_dev==1]))


47972


In [203]:
from  sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
num_features = X_train.select_dtypes(include='number').columns.tolist()
cat_features = X_train.select_dtypes(include='object').columns.tolist()
num_pipeline = Pipeline(steps=[
    ('imputer' , SimpleImputer(strategy='mean')),
    ('standard_scaler' , StandardScaler() )
    ])
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehotencoder' , OneHotEncoder(sparse_output=False)),
    ('standard_scaler' , StandardScaler())
])
preprocessing = ColumnTransformer(transformers=[
    ('num', num_pipeline , num_features),
    ('cat' , cat_pipeline , cat_features)
])
def transform_data(X_train):
    X_train_tr = preprocessing.fit_transform(X_train)
    return X_train_tr


In [204]:
X_dev_tr = transform_data(X_dev )
X_test_tr = transform_data(X_test)

In [205]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
param_grid_decision_tree = {
    'criterion': ['gini', 'entropy'],  # Measure of quality of a split
    'splitter': ['best', 'random'],  # Strategy used to choose the split at each node
    'max_depth': [ 15 , 30 , 40 , 50 ],  # Maximum depth of the tree  # Minimum number of samples required to split an internal nod  # Minimum number of samples required to be at a leaf node
    'max_features': [ 11 , 15 , 13]  # Number of features to consider when looking for the best split
}
grid_search_decision_tree = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=param_grid_decision_tree,
    scoring='accuracy',  # Scoring metric set to F1
    cv=5
)
grid_search_decision_tree.fit(X_dev_tr , y_dev)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [15, 30, 40, 50],
                         'max_features': [11, 15, 13],
                         'splitter': ['best', 'random']},
             scoring='accuracy')

In [206]:
grid_search_decision_tree.best_params_

{'criterion': 'entropy',
 'max_depth': 30,
 'max_features': 15,
 'splitter': 'best'}

In [210]:
from sklearn.metrics import classification_report
y_pred = grid_search_decision_tree.predict(X_test_tr)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00     47908
           1       0.01      1.00      0.01       341

    accuracy                           0.01     48249
   macro avg       0.50      0.50      0.01     48249
weighted avg       0.99      0.01      0.00     48249



              precision    recall  f1-score   support

           0       0.99      1.00      1.00     19160
           1       0.00      0.00      0.00       140

    accuracy                           0.99     19300
   macro avg       0.50      0.50      0.50     19300
weighted avg       0.99      0.99      0.99     19300



d:\github\backorder_prediction_model\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\github\backorder_prediction_model\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\github\backorder_prediction_model\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tree f1 score 0.0


In [144]:
param_grid_random_forest = {
    'n_estimators': [100, 150],  # Number of trees in the forest
    'criterion': ['gini', 'entropy'],  # Measure of quality of a split
    'max_depth': [5, 3, 2 , 7  , 10],  # Maximum depth of the tree  # Minimum number of samples required to split an internal node  # Minimum number of samples required to be at a leaf node
    'max_features': [5 , 7 , 3, 10 ,11 ]  # Number of features to consider when looking for the best split
}
grid_search_random_forest = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid_random_forest,
    scoring='accuracy',  # Scoring metric set to F1
    cv=5
)
grid_search_random_forest.fit(X_train_tr , y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 3, 2, 7, 10],
                         'max_features': [5, 7, 3, 10, 11],
                         'n_estimators': [100, 150]},
             scoring='accuracy')

In [147]:
grid_search_random_forest.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 7,
 'n_estimators': 150}

In [145]:
y_pred = grid_search_random_forest.predict(X_test_tr)
print(classification_report(y_pred , y_test))

              precision    recall  f1-score   support

           0       0.93      0.66      0.77     27118
           1       0.52      0.88      0.65     11334

    accuracy                           0.72     38452
   macro avg       0.72      0.77      0.71     38452
weighted avg       0.81      0.72      0.73     38452



In [ ]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier()

# Define the parameter grid for GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 5, 10],
    'gamma': [0, 0.1, 0.2, 0.3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0, 0.1, 0.5, 1]
}
xgb_grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, scoring='accuracy', cv=5)
xgb_grid_search.fit(X_train_tr, y_train)


from sklearn.ensemble import GradientBoostingClassifier


# Define the GBClassifier
gb_classifier = GradientBoostingClassifier()

# Define the hyperparameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],              # Number of boosting stages to be run
    'learning_rate': [0.1, 0.01, 0.001],           # Step size shrinkage used in update to prevent overfitting
    'max_depth': [3, 5, 7],                        # Maximum depth of the individual estimators
    'min_samples_split': [2, 5, 10],               # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],                 # Minimum number of samples required to be at a leaf node
    'subsample': [0.8, 0.9, 1.0]                   # Fraction of samples used for fitting the individual base learners
}

# Create the GridSearchCV instance
grid_search_gb = GridSearchCV(estimator=gb_classifier, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV to the data
grid_search_gb.fit(X_train_tr, y_train)

In [ ]:
y_pred = xgb_grid_search.predict(X_test_tr)
print(classification_report(y_pred , y_test))

In [ ]:
y_pred = grid_search_gb.predict(X_test_tr)
print(classification_report(y_pred , y_test))

In [82]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

# Assuming you have your feature matrix 'X' and target vector 'y'
# Replace X and y with your actual data
# X should be a pandas DataFrame or NumPy array containing the features
# y should be a pandas Series or NumPy array containing the target variable

# Create a list of models for feature ranking
models = [
    ('Logistic Regression', LogisticRegression(max_iter=100000000)),
    ('Random Forest', RandomForestClassifier()),
    ('Decision Tree', DecisionTreeClassifier())
]

# Set the number of features you want to select
num_features_to_select = 5

# Loop through each model and perform feature selection
for model_name, model in models:
    rfe_selector = RFE(estimator=model, n_features_to_select=num_features_to_select)
    rfe_selector = rfe_selector.fit(X_train_tr, y_train)

    selected_feature_indices = rfe_selector.get_support(indices=True)
    selected_feature_names = X_train.columns[selected_feature_indices]
    
    print(f"Selected Features for {model_name}:")
    print(selected_feature_names)


Selected Features for Logistic Regression:
Index(['national_inv', 'in_transit_qty', 'sales_3_month', 'sales_9_month',
       'pieces_past_due'],
      dtype='object')
Selected Features for Random Forest:
Index(['forecast_6_month', 'forecast_9_month', 'sales_9_month',
       'pieces_past_due', 'perf_6_month_avg'],
      dtype='object')
Selected Features for Decision Tree:
Index(['forecast_3_month', 'forecast_9_month', 'sales_9_month',
       'pieces_past_due', 'perf_6_month_avg'],
      dtype='object')
